In [1]:
import sys
import SimpleITK as sitk
import os
import numpy as np
%matplotlib inline 
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from ipywidgets import interact, fixed
import math
import glob
from scipy.interpolate import RegularGridInterpolator, interpn

from skimage import measure

import pylab as pl
import trimesh
from stl import mesh
import re

from scipy.spatial import KDTree

In [2]:
#prepare data set
def centeroidnp(arr):
    """get the centroid of a point cloud"""
    length = arr.shape[0]
    sum_x = np.sum(arr[:, 0])
    sum_y = np.sum(arr[:, 1])
    sum_z = np.sum(arr[:, 2])
    return math.ceil(sum_x/length), \
            math.ceil(sum_y/length), \
            math.ceil(sum_z/length)

In [3]:
def create_7D(source_pc, source_center, target_center):
    """create a 7D pointcloud as explained in Fu et al."""
    v_s = np.zeros((len(source_pc), 7))
    for i in range(len(v_s)):
        v_ss = source_center - source_pc[i,:3]
        v_st = target_center - source_pc[i,:3]
        v_s[i,:3] = v_ss 
        v_s[i,3:6] = v_st
        v_s[i,6] = source_pc[i,3]
    return v_s


In [4]:
def set_position_and_orientation(files_deform, files_regular):
    """
    set the positioning and orientation of the deformed images so that they align
    NOTE: this should be done in the begining when the initial deformed mhd files are created
    """
    for d ,f in zip(files_deform, files_regular):
        with open(d, 'a+') as deformed:
            with open(f, 'r') as regular:
                for r in regular.readlines():
                    if "Position" in r: #or "Orientation" in r:
                        deformed.write(r)

# Make connections between vertebrae bodies and laminas with facets for XML scene in SOFA framework

In [5]:

def dist_pts(a, b):
    return np.linalg.norm(a-b)

def min_dist(points, p):
    min_=min([dist_pts(a[:3],p) for a in points])
    return [dist_pts(a[:3],p) for a in points].index(min_)

def intersect2d(X, Y):
        """
        Function to find intersection of two 2D arrays.
        Returns index of rows in X that are common to Y.
        """
        X = np.tile(X[:,:,None], (1, 1, Y.shape[0]) )
        Y = np.swapaxes(Y[:,:,None], 0, 2)
        Y = np.tile(Y, (X.shape[0], 1, 1))
        eq = np.all(np.equal(X, Y), axis = 1)
        eq = np.any(eq, axis = 1)
        return np.nonzero(eq)[0]


In [6]:
def get_bbox(position):
    """ 
    Gets the bounding box of the object defined by the given vertices.

    Arguments
    -----------
    position : list
    List with the coordinates of N points (position field of Sofa MechanicalObject).

    Returns
    ----------
    xmin, xmax, ymin, ymax, zmin, zmax : floats
    min and max coordinates of the object bounding box.
    """
    points_array = np.asarray(position)
    m = np.min(points_array, axis=0)
    xmin, ymin, zmin = m[0], m[1], m[2]

    m = np.max(points_array, axis=0)
    xmax, ymax, zmax = m[0], m[1], m[2]

    return xmin, xmax, ymin, ymax, zmin, zmax

def get_indices_in_bbox( positions, bbox ):
    """
    Get the indices of the points falling within the specified bounding box.

    Arguments
    ----------
    positions (list):
    N x 3 list of points coordinates.
    bbox (list):
    [xmin, ymin, zmin, xmax, ymax, zmax] extremes of the bounding box.

    Returns
    ----------
    indices:
    List of indices of points enclosed in the bbox.

    """
    # bbox is in the format (xmin, ymin, zmin, xmax, ...)
    assert len(bbox) == 6
    indices = []
    for i, x in enumerate( positions ):
        if x[0] >= bbox[0] and x[0] <= bbox[3] and x[1] >= bbox[1] and x[1] <= bbox[4] and x[2] >= bbox[2] and x[2] <= bbox[5]:
            indices.append( i )
    return indices
        

def print_stiff_springs(vert1,vert2, bbox_v1_v2,bbox_v2_v1, s, d):
    """
    vert1 and vert2: two adjecent vertebrae
    bbox_v1_v2 and bbox_v2_v1 are the bounding boxes representing area where the 
    springs are found on the closer sides of two adjecent vertebrae
    """
    idx1 = get_indices_in_bbox(vert1, bbox_v1_v2)[::5]
    idx2 = get_indices_in_bbox(vert2, bbox_v2_v1)[::5]
    print("SPRINGS: ")
    np.random.shuffle(idx1)
    np.random.shuffle(idx2)
    for i,j in zip(idx1,idx2):
        print("{0} {1} {2} {3} {4}  ".format(i,j,s,d,dist_pts(vert1[i],vert2[j])), end='')
        
def print_positions(vert1, bbox_v1_t12):
    """
    this function is used for seting the fixed points on L1 and L5 simulating
    connection with T12 and S1 respectively
    """
    
    idx1 = get_indices_in_bbox(vert1, bbox_v1_t12)[::5]
    print("POSITIONS:")
    for i in idx1:
        print("{0} {1} {2}  ".format(vert1[i][0],vert1[i][1],vert1[i][2]), end='')
    print("Indexes for fixed constraint")
    for i,_ in enumerate(idx1):
        print(i, end=" ")
    print("SPRINGS: ")
    for i,j in enumerate(idx1):
        print("{0} {1} {2} {3} {4}  ".format(i,j,1000,10,0.00001), end='')



## uncomment the block below to print the connections and vertices for the springs in SOFA framework change paths and bounding boxes accordingly

In [7]:
def bboxes(values):
    return [np.min(values[:,0]), np.min(values[:,1]) ,np.min(values[:,2]),
      np.max(values[:,0]), np.max(values[:,1]),np.max(values[:,2])]

In [39]:
#print bounding boxes
for i in range(14,17):
    print("#################################  " + str(i))
    vert1 = np.loadtxt("/Users/janelameski/Downloads/new_spines_Jane/spine"+str(i)+"/v1.txt")[:,:3]
    vert2 = np.loadtxt("/Users/janelameski/Downloads/new_spines_Jane/spine"+str(i)+"/v2.txt")[:,:3]
    vert3 = np.loadtxt("/Users/janelameski/Downloads/new_spines_Jane/spine"+str(i)+"/v3.txt")[:,:3]
    vert4 = np.loadtxt("/Users/janelameski/Downloads/new_spines_Jane/spine"+str(i)+"/v4.txt")[:,:3]
    vert5 = np.loadtxt("/Users/janelameski/Downloads/new_spines_Jane/spine"+str(i)+"/v5.txt")[:,:3] 

    values = np.loadtxt("/Users/janelameski/Downloads/new_spines_Jane/spine"+str(i)+"/v1_t12.txt")
    print(bboxes(values))
    values = np.loadtxt("/Users/janelameski/Downloads/new_spines_Jane/spine"+str(i)+"/v2_v1.txt")
    print(bboxes(values))
    print(bboxes(values))
    values = np.loadtxt("/Users/janelameski/Downloads/new_spines_Jane/spine"+str(i)+"/v3_v2.txt")
    print(bboxes(values))
    print(bboxes(values))
    values = np.loadtxt("/Users/janelameski/Downloads/new_spines_Jane/spine"+str(i)+"/v4_v3.txt")
    print(bboxes(values))
    print(bboxes(values))

    values = np.loadtxt("/Users/janelameski/Downloads/new_spines_Jane/spine"+str(i)+"/v5_v4.txt")
    print(bboxes(values))
    print(bboxes(values))

    values = np.loadtxt("/Users/janelameski/Downloads/new_spines_Jane/spine"+str(i)+"/v5_s1.txt")
    print(bboxes(values))

    values = np.loadtxt("/Users/janelameski/Downloads/new_spines_Jane/spine"+str(i)+"/facet12.txt")
    print(bboxes(values))
    print(bboxes(values))

    values = np.loadtxt("/Users/janelameski/Downloads/new_spines_Jane/spine"+str(i)+"/facet23.txt")
    print(bboxes(values))
    print(bboxes(values))

    values = np.loadtxt("/Users/janelameski/Downloads/new_spines_Jane/spine"+str(i)+"/facet34.txt")
    print(bboxes(values))
    print(bboxes(values))

    values = np.loadtxt("/Users/janelameski/Downloads/new_spines_Jane/spine"+str(i)+"/facet45.txt")
    print(bboxes(values))
    print(bboxes(values))



#################################  14
[-0.0368868, 0.0618011, -0.319603, 0.0101893, 0.0959398, -0.310338]
[-0.0338871, 0.0581104, -0.352957, 0.0104191, 0.0894811, -0.338596]
[-0.0338871, 0.0581104, -0.352957, 0.0104191, 0.0894811, -0.338596]
[-0.0343105, 0.0484098, -0.376846, 0.0152943, 0.0816258, -0.362949]
[-0.0343105, 0.0484098, -0.376846, 0.0152943, 0.0816258, -0.362949]
[-0.0374403, 0.0387444, -0.406647, 0.0248827, 0.0724218, -0.385801]
[-0.0374403, 0.0387444, -0.406647, 0.0248827, 0.0724218, -0.385801]
[-0.0215067, 0.0323159, -0.436645, 0.0274513, 0.0646093, -0.420787]
[-0.0215067, 0.0323159, -0.436645, 0.0274513, 0.0646093, -0.420787]
[-0.0237078, 0.036478, -0.466633, 0.0300961, 0.066926, -0.452848]
[-0.0284107, 0.0935076, -0.364092, 0.0138307, 0.118086, -0.34585]
[-0.0284107, 0.0935076, -0.364092, 0.0138307, 0.118086, -0.34585]
[-0.0300255, 0.0892332, -0.393064, 0.0199284, 0.106331, -0.374911]
[-0.0300255, 0.0892332, -0.393064, 0.0199284, 0.106331, -0.374911]
[-0.0266599, 0.077

In [16]:
vert1 = np.loadtxt("/Users/janelameski/Downloads/new_spines_Jane/spine19/v1.txt")[:,:3]
vert2 = np.loadtxt("/Users/janelameski/Downloads/new_spines_Jane/spine19/v2.txt")[:,:3]
vert3 = np.loadtxt("/Users/janelameski/Downloads/new_spines_Jane/spine19/v3.txt")[:,:3]
vert4 = np.loadtxt("/Users/janelameski/Downloads/new_spines_Jane/spine19/v4.txt")[:,:3]
vert5 = np.loadtxt("/Users/janelameski/Downloads/new_spines_Jane/spine19/v5.txt")[:,:3] 

values = np.loadtxt("/Users/janelameski/Downloads/new_spines_Jane/spine19/v1_t12.txt")
bbox_v1_t12 = bboxes(values)
values = np.loadtxt("/Users/janelameski/Downloads/new_spines_Jane/spine19/v2_v1.txt")
bbox_v1_v2 = bboxes(values)
bbox_v2_v1 = bboxes(values)
values = np.loadtxt("/Users/janelameski/Downloads/new_spines_Jane/spine19/v3_v2.txt")
bbox_v2_v3 = bboxes(values)
bbox_v3_v2 = bboxes(values)
values = np.loadtxt("/Users/janelameski/Downloads/new_spines_Jane/spine19/v4_v3.txt")
bbox_v3_v4 = bboxes(values)
bbox_v4_v3 = bboxes(values)
values = np.loadtxt("/Users/janelameski/Downloads/new_spines_Jane/spine19/v5_v4.txt")
bbox_v4_v5 = bboxes(values)
bbox_v5_v4 = bboxes(values)
values = np.loadtxt("/Users/janelameski/Downloads/new_spines_Jane/spine19/v5_s1.txt")
bbox_v5_s1 = bboxes(values)
bbox_v5_s1[0]+=0.012
bbox_v5_s1[3]-=0.012

values = np.loadtxt("/Users/janelameski/Downloads/new_spines_Jane/spine19/facet12.txt")
bbox_bone_v1_v2 = bboxes(values)
bbox_bone_v2_v1 = bboxes(values)
values = np.loadtxt("/Users/janelameski/Downloads/new_spines_Jane/spine19/facet23.txt")
bbox_bone_v2_v3 = bboxes(values)
bbox_bone_v3_v2 = bboxes(values)
values = np.loadtxt("/Users/janelameski/Downloads/new_spines_Jane/spine19/facet34.txt")
bbox_bone_v3_v4 = bboxes(values)
bbox_bone_v4_v3 = bboxes(values)
values = np.loadtxt("/Users/janelameski/Downloads/new_spines_Jane/spine19/facet45.txt")
bbox_bone_v4_v5 = bboxes(values)
bbox_bone_v5_v4 = bboxes(values)


print_positions(vert1, bbox_v1_t12)
print()

print_positions(vert5, bbox_v5_s1)
print()
# format(i,j,500,3,dist_pts(vert1[i],vert2[j])), end='')
print_stiff_springs(vert1, vert2, bbox_v1_v2, bbox_v2_v1,500,3)
print()
print_stiff_springs(vert2, vert3, bbox_v2_v3, bbox_v3_v2,500,3)
print()
print_stiff_springs(vert3, vert4, bbox_v3_v4, bbox_v4_v3,500,3)
print()
print_stiff_springs(vert4, vert5, bbox_v4_v5, bbox_v5_v4,500,3)
# format(i,j,8000,500,dist_pts(vert1[i],vert2[j])), end='')
print()
print()
print_stiff_springs(vert1, vert2, bbox_bone_v1_v2, bbox_bone_v2_v1,8000,500)
print()
print_stiff_springs(vert2, vert3, bbox_bone_v2_v3, bbox_bone_v3_v2,8000,500)
print()
print_stiff_springs(vert3, vert4, bbox_bone_v3_v4, bbox_bone_v4_v3,8000,500)
print()
print_stiff_springs(vert4, vert5, bbox_bone_v4_v5, bbox_bone_v5_v4,8000,500)

POSITIONS:
-0.019 0.141508 -0.34497  -0.017 0.140508 -0.34497  -0.02 0.141533 -0.344995  -0.017025 0.141533 -0.34597  -0.016025 0.140533 -0.34597  -0.015025 0.139533 -0.34597  -0.021 0.142533 -0.345995  -0.018025 0.142533 -0.34697  -0.021 0.140533 -0.345995  -0.020975 0.139533 -0.34697  -0.022 0.144508 -0.34797  -0.02 0.143533 -0.346995  -0.019 0.143508 -0.34797  -0.021975 0.141533 -0.34797  -0.016025 0.140533 -0.34797  -0.015 0.140508 -0.34797  -0.008 0.139533 -0.346995  -0.01 0.139508 -0.34797  -0.023 0.144533 -0.347995  -0.020025 0.144533 -0.34897  -0.019 0.144508 -0.34897  -0.022 0.141558 -0.34897  -0.016 0.140533 -0.347995  -0.01 0.140533 -0.347995  -0.007 0.140533 -0.347995  -0.015 0.140508 -0.34897  -0.011975 0.139533 -0.34897  -0.006 0.139533 -0.347995  -0.012 0.139508 -0.34897  -0.022 0.145533 -0.348995  -0.023975 0.144533 -0.34997  -0.022975 0.143533 -0.34997  -0.017 0.144508 -0.34997  -0.012 0.143533 -0.348995  -0.01 0.144508 -0.34997  -0.016 0.142533 -0.348995  -0.013 0.142

14238 984 500 3 0.026776168348738767  14680 2188 500 3 0.022842793196104553  11399 3485 500 3 0.028649593382803866  11821 979 500 3 0.014169911820473696  12871 2398 500 3 0.02141929973178395  13380 2861 500 3 0.009098703259256258  11846 2821 500 3 0.020416395396837317  12755 813 500 3 0.010839059045876601  14591 2238 500 3 0.01792747614696505  13725 2250 500 3 0.021945168055861428  13796 2908 500 3 0.022601707922190316  11444 2255 500 3 0.01584232309353648  12285 968 500 3 0.013663396393283771  13749 918 500 3 0.024705124994624114  12270 3928 500 3 0.017076404523201007  14361 477 500 3 0.03833012393666371  13425 3475 500 3 0.019206311239798254  13375 2743 500 3 0.010202215494685486  14221 3391 500 3 0.026901457228187483  13392 2104 500 3 0.021229423002050725  13894 2297 500 3 0.010133469346674912  13739 505 500 3 0.025987581668943355  12431 3372 500 3 0.026496799825639347  11379 1511 500 3 0.01360261743195035  13877 160 500 3 0.03592723759211109  10918 2720 500 3 0.03418077238741102  1

15096 2576 8000 500 0.028531428302838276  14915 4008 8000 500 0.01562435281859701  16079 6606 8000 500 0.021615698022502093  15136 7390 8000 500 0.017731564833369913  16488 1100 8000 500 0.029088840480502143  16693 5317 8000 500 0.012648735549742034  15520 602 8000 500 0.0030008599600947583  16008 229 8000 500 0.010227922614099117  15983 340 8000 500 0.013570609437313414  16099 1663 8000 500 0.03231111265493654  16339 400 8000 500 0.03421955568735514  16334 4454 8000 500 0.026861563458592655  15201 1753 8000 500 0.02914739767869509  16698 355 8000 500 0.014177376379288243  16013 2481 8000 500 0.015783548428664584  15811 3190 8000 500 0.008832553481298605  15186 4839 8000 500 0.016623320710375533  15359 5275 8000 500 0.01414515183729396  15344 6451 8000 500 0.020117783202927727  15505 5988 8000 500 0.013598691150254166  14457 6516 8000 500 0.02957947365008547  16782 5270 8000 500 0.014999333033171847  16887 3561 8000 500 0.014495612834233684  15958 5295 8000 500 0.019415656147305872  15

## Take points for Biomechanical constraint

In [93]:
def find_nearest_vector(array, value):
    idx = np.array([np.linalg.norm(x+y+z) for (x,y,z) in np.abs(array[:,:3]-value[:3])]).argmin()
    return int(idx)

In [9]:
vert1 = np.loadtxt("/Users/janelameski/Downloads/new_spines_Jane/spine11/v1.txt")[:,:3]
# vert2 = np.loadtxt("DataJane/Spine10/v2.txt")[:,:3]
# vert3 = np.loadtxt("DataJane/Spine10/v3.txt")[:,:3]
# vert4 = np.loadtxt("DataJane/Spine10/v4.txt")[:,:3]
# vert5 = np.loadtxt("DataJane/Spine10/v5.txt")[:,:3]

# verts = [vert1,vert2,vert3,vert4,vert5]
# #bounding boxes of the vertebrae, produced manually and saved in the above cell for 
# #every spine separately
# bbox_v1_t12 = [-0.0239, 0.2275, -0.0591, 0.0629, 0.2764, -0.0442]
# bbox_v1_v2 = [-0.0239, 0.2275, -0.08, 0.0629, 0.272, -0.062]
# bbox_v2_v1 = [-0.0239, 0.2275, -0.087, 0.0629, 0.266, -0.062]
# bbox_v2_v3 = [-0.0239, 0.2265, -0.11, 0.0629, 0.262, -0.09]
# bbox_v3_v2 = [-0.0239, 0.2, -0.115, 0.0629, 0.253, -0.09]
# bbox_v3_v4 = [-0.0239, 0.2, -0.14, 0.0629, 0.248, -0.122]
# bbox_v4_v3 = [-0.0239, 0.2, -0.15, 0.0629, 0.245, -0.122]
# bbox_v4_v5 = [-0.0239, 0.2, -0.18, 0.0629, 0.245, -0.16]
# bbox_v5_v4 = [-0.0239, 0.19, -0.187, 0.0629, 0.24, -0.16]
# bbox_v5_s1 = [-0.0239, 0.19, -0.214, 0.0629, 0.24, -0.195]

# bboxes = [bbox_v1_t12, bbox_v1_v2,
#           bbox_v2_v1, bbox_v2_v3, 
#           bbox_v3_v2, bbox_v3_v4,
#           bbox_v4_v3, bbox_v4_v5,
#           bbox_v5_v4, bbox_v5_s1]

# len_v = 0
# with open("Spine10_biomechanical.txt", "w+") as file:
#     indices1 = get_indices_in_bbox(verts[0], bboxes[1])
#     indices2 = get_indices_in_bbox(verts[1], bboxes[2])
#     indices3 = get_indices_in_bbox(verts[1], bboxes[3])
#     indices4 = get_indices_in_bbox(verts[2], bboxes[4])
#     indices5 = get_indices_in_bbox(verts[2], bboxes[5])
#     indices6 = get_indices_in_bbox(verts[3], bboxes[6])
#     indices7 = get_indices_in_bbox(verts[3], bboxes[7])
#     indices8 = get_indices_in_bbox(verts[4], bboxes[8])

    
#     a1 = find_nearest_vector(verts[0],np.mean(verts[0][indices1], axis=0))
#     a2 = find_nearest_vector(verts[1],np.mean(verts[1][indices2], axis=0))
#     a3 = find_nearest_vector(verts[1],np.mean(verts[1][indices3], axis=0))
#     a4 = find_nearest_vector(verts[2],np.mean(verts[2][indices4], axis=0))
#     a5 = find_nearest_vector(verts[2],np.mean(verts[2][indices5], axis=0))
#     a6 = find_nearest_vector(verts[3],np.mean(verts[3][indices6], axis=0))
#     a7 = find_nearest_vector(verts[3],np.mean(verts[3][indices7], axis=0))
#     a8 = find_nearest_vector(verts[4],np.mean(verts[4][indices8], axis=0))
    
#     file.write("{0} {1} {2} {3} {4} {5} {6} {7}".format(a1,a2,a3,a4,a5,a6,a7,a8))
    #SANITY CHECK
#     file.write("{0} {1} {2}\n{3} {4} {5}\n{6} {7} {8}\n{9} {10} {11}\n{12} {13} {14}\n{15} {16} {17}\n{18} {19} {20}\n{21} {22} {23}\n"
#                                 .format(verts[0][a1][0],
#                                         verts[0][a1][1],
#                                         verts[0][a1][2],
#                                         verts[1][a2][0],
#                                         verts[1][a2][1],
#                                         verts[1][a2][2],
#                                          verts[1][a3][0],
#                                         verts[1][a3][1],
#                                         verts[1][a3][2],
#                                         verts[2][a4][0],
#                                         verts[2][a4][1],
#                                         verts[2][a4][2],
#                                          verts[2][a5][0],
#                                         verts[2][a5][1],
#                                         verts[2][a5][2],
#                                         verts[3][a6][0],
#                                         verts[3][a6][1],
#                                         verts[3][a6][2],
#                                          verts[3][a7][0],
#                                         verts[3][a7][1],
#                                         verts[3][a7][2],
#                                         verts[4][a8][0],
#                                         verts[4][a8][1],
#                                         verts[4][a8][2],
#                                                          ))

# vtu to txt

In [51]:
#change paths accordingly
path_vtu_files = "/Users/janelameski/Desktop/jane/sofa/SOFAZIPPED/install/bin/vtu_files/"
path_txt_files = "/Users/janelameski/Desktop/jane/sofa/SOFAZIPPED/install/bin/txtFiles/"
def vtu2txt(path_vtu_files, path_txt_files):
    list_files = []
    #check lines which start anything but numbers(in the vtu files the rows 
    #with numbers are the rows containing the point cloud)
    regex = re.compile("^ *<|^  +\d|^\t<|^\t\d|^\t-|^ +-")
    
    #put all vtu files in list_files
    for file in os.listdir(path_vtu_files):
        if file.endswith(".vtu"):
            list_files.append(file)
    #read all files one by one
    for file in list_files:
        with open(path_vtu_files + file, "r") as f:
            lines = f.readlines()
        #filter them using the regex above
        filtered = [i for i in lines if not regex.match(i)]
        #write them in a file with same name but ending txt
        with open(path_txt_files + file[:-3]+"txt", "w+") as f:
            for l in filtered:
                
                x,y,z = l.split(" ")
                f.write("{0} {1} {2}\n".format(float(x)*1e+3,float(y)*1e+3,float(z)*1e+3))

##  uncomment below to change vtu to point clouds txt

In [52]:
vtu2txt(path_vtu_files, path_txt_files)

## Convert vtu to obj; Uncomment to run

In [98]:
import meshio
#change path accordingly
path_to_vtu = "/Users/janelameski/Desktop/jane/sofa/SOFAZIPPED/install/bin/"

for file in os.listdir(path_to_vtu + "vtu_files/"):
    if file.endswith(".vtu"):

        mesh_vtu = meshio.read(path_to_vtu + "vtu_files/" + file)
    
        mesh = meshio.Mesh(
        mesh_vtu.points*1e3,
        mesh_vtu.cells,
        # Optionally provide extra data on points, cells, etc.
        mesh_vtu.point_data,
        # Each item in cell data must match the cells array
        mesh_vtu.cell_data,
        )
        mesh.write(path_to_vtu + "obj_files/" + file[:-3]+ "obj")


## obj to mhd is done using ImFusion

## raycast

In [99]:
def raycast_spine1(image):# spine10, spine7 , spine6 , spine 5, spine4 , spine3 , spine2 , spine 1, 
    rays = np.zeros_like(np.squeeze(np.squeeze(image)))
    for i in range(image.shape[1]):
        for j in range(image.shape[0]-1, 0, -1):
            if image[j, i] != 0:
                rays[j, i] = 1
                break
    return rays
def raycast_spine8(image):# spine 8
    rays = np.zeros_like(np.squeeze(np.squeeze(image)))
    for i in range(image.shape[1]):
        for j in range(image.shape[0]):
            if image[j, i] != 0:
                rays[j, i] = 1
                break
    return rays


def raycast_files(files_path, spine_id):
    for path in os.listdir(files_path):
        if path.endswith(".mhd") and path.startswith(spine_id):
            img_mhd = sitk.ReadImage(files_path + path)
            im = sitk.GetArrayFromImage(img_mhd)

            raycasted = np.zeros_like(im)
            
            if "e1" in spine_id or "e2" in spine_id or "e3" in spine_id or "e4" in spine_id or "e6" in spine_id or "e7" in spine_id or "e10" in spine_id:
                for i in range(im.shape[0]):
                    raycasted[i,...] = raycast_spine1(im[i,...])
            else:
                if "e5" in spine_id or "e9" in spine_id:
                    for i in range(im.shape[2]):
                        raycasted[...,i] = raycast_spine1(im[...,i])#spine 5, 9
                elif "e8" in spine_id :
                    for i in range(im.shape[2]):
                        raycasted[...,i] = raycast_spine8(im[...,i])#spine 8,
            
            raycasted_img = sitk.GetImageFromArray(raycasted)
            sitk.WriteImage(raycasted_img,  files_path + "raycasted_" +path[:-3] + "mhd")
        

## Uncomment the cell below to create raycasted mhd files 

In [100]:
path_to_label_maps = "/Users/janelameski/Desktop/labelMaps/"
list_of_spine_ids = [i[:-4] for i in os.listdir(path_to_label_maps) if i.startswith("spine") and i.endswith("mhd")]
for spine_id in list_of_spine_ids:
    raycast_files(path_to_label_maps, spine_id)
    list_raycasted = sorted([path_to_label_maps + i for i in os.listdir(path_to_label_maps) if i.startswith("raycasted_"+spine_id) and i.endswith(".mhd")])
    list_non_raycasted = sorted([path_to_label_maps + i for i in os.listdir(path_to_label_maps) if i.startswith(spine_id) and i.endswith(".mhd")])
    set_position_and_orientation(list_raycasted, list_non_raycasted)



## mhd to .txt is done using ImFusion

# Prepare spine data .txt to .npz

In [113]:
list_files = "/Users/janelameski/Desktop/jane/sofa/SOFAZIPPED/install/bin/" + "txtFiles/"
def obtain_file_id(p):
    """
    IMPORTANT: make sure that the path to files folder (p) does not contain "_" except in the name of the 
    files themselves
    """
    splits = p.split("_")
    #dealing with Source point clouds "spine1_vert10.txt"
    if len(splits)==2:
        if splits[0].endswith("0"):
            s_id = splits[0][-7:] + splits[1][5]
        else:
            s_id = splits[0][-6:] + splits[1][5]
    #this is the case only for several target files such as spine1_vert2_1.txt
    elif len(splits)==3 and splits[2].endswith(".txt"):
        s_id = splits[0][-6:] + "_" +splits[2][0]    
    #for all other target files such as "spine1_vert5_1_0"
    else:
        if splits[0].endswith("0"):
            s_id = splits[0][-7:] + "_"+splits[2]+"_"+splits[3][0]
        else:
            s_id = splits[0][-6:] + "_"+splits[2]+"_"+splits[3][0]
    return s_id

def obtain_indices_raycasted_original_pc(spine_target, r_target):
    """
    find indices in spine_target w.r.t. r_target such that they are the closest points between the two 
    point clouds
    """
    kdtree=KDTree(spine_target[:,:3])
    dist,points=kdtree.query(r_target[:,:3],1)

    return list(set(points))

def create_source_target_with_vertebra_label(source_pc, target_pc, vert):
    """
    source_pc: source point cloud
    target_pc: target point cloud
    vert: [1-5] for [L1-L5] vertebra respectively
    
    this function is to create source and target point clouds with label for each vertebra
    """
    
    source = np.ones((source_pc.shape[0], source_pc.shape[1]+1))
    source[:, :3]=source_pc
    source[:, 3] = source[:, 3]*vert
    target = np.ones((target_pc.shape[0], target_pc.shape[1]+1))
    target[:, :3]=target_pc
    target[:, 3]= target[:, 3]*vert
    
    return source, target

def create_source_target_flow_spine(source_pc, target_pc):
    """
    source_pc: source point cloud
    target_pc: target point cloud
    vert: [1-5] for [L1-L5] vertebra respectively
    
    this function is to create source and target point clouds with 7D
    where the point clouds are centered.
    """
    
#     source_pc, target_pc = create_source_target_with_vertebra_label(source_pc, target_pc, vert)

    centroid_source = centeroidnp(source_pc)
    centroid_target = centeroidnp(target_pc)
    
    source_7d = create_7D(source_pc, centroid_source, centroid_target)
    target_7d = create_7D(target_pc, centroid_source, centroid_target)
    
    flow = target_7d[:,:3]-source_7d[:,:3]
    
    return source_7d, target_7d, flow



def write_source_target_flow_as_npz(pa, num):
    """
    USE THIS METHOD TO CREATE FULL SPINE DATA AS .npz
    
    pa: path to vertebrae data set with name of files: 
    "spine{num_spine}_vert{num_vert}_{SOFA_experiment}_{Nx20_iterations}.txt"
    num: which patient does the spine belong to labeled from 1-10
    
    returns a list of spine point clouds containing flow source target, 
    and another list with the constraints
    """
    source_rgx = re.compile("^spine"+str(num)+"_vert.0")
    l1_rgx = re.compile("^spine"+str(num)+"_vert1_")
    l2_rgx = re.compile("^spine"+str(num)+"_vert2_")
    l3_rgx = re.compile("^spine"+str(num)+"_vert3_")
    l4_rgx = re.compile("^spine"+str(num)+"_vert4_")
    l5_rgx = re.compile("^spine"+str(num)+"_vert5_")
    
    source_list = []
    #if the path to the biomechanical constraints is different, add it accordingly 
    constraint_list = np.loadtxt("Spine"+str(num)+"_biomechanical.txt")
    l1=[]
    l2=[]
    l3=[]
    l4=[]
    l5=[]
    
    #list the files matching the regex above to their name
    for file in os.listdir(pa):
        if file.endswith(".txt") and source_rgx.match(file):
            source_list.append(os.path.join(pa, file))
        if file.endswith(".txt") and l1_rgx.match(file):
            l1.append(os.path.join(pa, file))
        if file.endswith(".txt") and l2_rgx.match(file):
            l2.append(os.path.join(pa, file))
        if file.endswith(".txt") and l3_rgx.match(file):
            l3.append(os.path.join(pa, file))
        if file.endswith(".txt") and l4_rgx.match(file):
            l4.append(os.path.join(pa, file))
        if file.endswith(".txt") and l5_rgx.match(file):
            l5.append(os.path.join(pa, file))
    
    source_list = sorted(source_list)
    targets = (sorted(l1),sorted(l2),sorted(l3),sorted(l4),sorted(l5))

    list_of_triples = []
    list_of_spines = []
    for i, t in enumerate(targets):
        
        for p in t:
            sour, tar = create_source_target_with_vertebra_label(np.loadtxt(source_list[i]), np.loadtxt(p), i+1)
            t_id = obtain_file_id(p)
            s_id = obtain_file_id(source_list[i])
            list_of_triples.append([sour, tar, t_id])

        list_of_spines.append(list_of_triples)
        list_of_triples=[]

    #for every vertebra in the combined files
    for t1,t2,t3,t4,t5 in zip(list_of_spines[0], list_of_spines[1], list_of_spines[2],
                         list_of_spines[3], list_of_spines[4]):
        
        #create the whole spines (source and target)
        spine_source = np.concatenate((t1[0],t2[0],t3[0],t4[0],t5[0]))
        spine_target = np.concatenate((t1[1],t2[1],t3[1],t4[1],t5[1]))
        
        source, target, flow = create_source_target_flow_spine(spine_source, spine_target)
    
        target_id = t1[2]
        
        np.savez_compressed(pa + target_id + ".npz",
                                    flow=flow,
                                    pc1=source,
                                    pc2=target,
                                    cstPts=constraint_list)


def combine_vertebrae_with_label(pa, num, raycasted_path):
    """
    USE THIS METHOD TO CREATE RAY CASTED SPINE DATA AS .npz 
    
    pa: path to vertebrae data set with name of files:
    "spine{num_spine}_vert{num_vert}_{SOFA_experiment}_{Nx20_iterations}.txt"
    num: which spine we are transforming
    raycasted_path: path where the raycasts are saved as
    """
    
    #select the file names according to whether they are source "spine1_vert10.txt" or target "spine1_vert1_1_0"
    source_rgx = re.compile("^spine"+str(num)+"_vert.0")
    l1_rgx = re.compile("^spine"+str(num)+"_vert1_")
    l2_rgx = re.compile("^spine"+str(num)+"_vert2_")
    l3_rgx = re.compile("^spine"+str(num)+"_vert3_")
    l4_rgx = re.compile("^spine"+str(num)+"_vert4_")
    l5_rgx = re.compile("^spine"+str(num)+"_vert5_")
    
    l1=[]
    l2=[]
    l3=[]
    l4=[]
    l5=[]
    
    source_list = []
    
    #list the files matching the regex above to their name
    for file in os.listdir(pa):
        if file.endswith(".txt") and source_rgx.match(file):
            source_list.append(os.path.join(pa, file))
        if file.endswith(".txt") and l1_rgx.match(file):
            l1.append(os.path.join(pa, file))
        if file.endswith(".txt") and l2_rgx.match(file):
            l2.append(os.path.join(pa, file))
        if file.endswith(".txt") and l3_rgx.match(file):
            l3.append(os.path.join(pa, file))
        if file.endswith(".txt") and l4_rgx.match(file):
            l4.append(os.path.join(pa, file))
        if file.endswith(".txt") and l5_rgx.match(file):
            l5.append(os.path.join(pa, file))
            
    source_list = sorted(source_list)
    targets = (sorted(l1),sorted(l2),sorted(l3),sorted(l4),sorted(l5))

    list_of_triplets = []
    list_of_spines = []
    

    #combine the files in a list of lists containing [source, target, source_id, target_id]
    for i, t in enumerate(targets):
        for p in t:
            sour, tar = create_source_target_with_vertebra_label(np.loadtxt(source_list[i]), np.loadtxt(p), i+1)
            t_id = obtain_file_id(p)
            s_id = obtain_file_id(source_list[i])
            list_of_triplets.append([sour,tar,s_id, t_id])
        list_of_spines.append(list_of_triplets)
        list_of_triplets = []
    spine_source = []
    spine_target = []
    
    #load the biomechanical constraint point indices
    bio_constraints = np.loadtxt("Spine"+str(num)+"_biomechanical.txt")
    
    #for every vertebra in the combined files
    for t1,t2,t3,t4,t5 in zip(list_of_spines[0], list_of_spines[1], list_of_spines[2],
                         list_of_spines[3], list_of_spines[4]):
        
        #create the whole spines (source and target)
        spine_source = np.concatenate((t1[0],t2[0],t3[0],t4[0],t5[0]))
        spine_target = np.concatenate((t1[1],t2[1],t3[1],t4[1],t5[1]))
        
        #create the flow
        spine_flow = spine_target[:,:3] - spine_source[:,:3]
        
        #the id of the spine
        source_id = t1[2]
        target_id = t1[3]
        
        #find the respective points for the biomechanical constraint point indices
        bio_constraints = [int(i) for i in bio_constraints]
        bio_points = [find_nearest_vector(spine_source, t1[0][bio_constraints[0]]),
                      find_nearest_vector(spine_source, t2[0][bio_constraints[1]]),
                      find_nearest_vector(spine_source, t2[0][bio_constraints[2]]),
                      find_nearest_vector(spine_source, t3[0][bio_constraints[3]]),
                      find_nearest_vector(spine_source, t3[0][bio_constraints[4]]),
                      find_nearest_vector(spine_source, t4[0][bio_constraints[5]]),
                      find_nearest_vector(spine_source, t4[0][bio_constraints[6]]),
                      find_nearest_vector(spine_source, t5[0][bio_constraints[7]])]
        

        #load the extracted point clouds from the .mhd ray casts
        r_source = np.loadtxt(raycasted_path + "raycasted_" + source_id + ".txt")
        r_target = np.loadtxt(raycasted_path + "raycasted_" + target_id + ".txt")
        
        raycasted_source_idcs = obtain_indices_raycasted_original_pc(spine_source, r_source)
        raycasted_target_idcs = obtain_indices_raycasted_original_pc(spine_target, r_target)
        #add the biomechanical constraint points in the raycasted_source because they are probably
        #not part of the point cloud
        raycasted_source_idcs.extend(bio_points)
        
        #select only the ray casted points from the original source and target
        raycasted_source = spine_source[raycasted_source_idcs, :]
        raycasted_target = spine_target[raycasted_target_idcs, :]
        raycasted_source_flow = np.zeros_like(raycasted_source)
        raycasted_source_flow[:,:3] = raycasted_source[:,:3] + spine_flow[raycasted_source_idcs, :]
        raycasted_source_flow[:,3] = raycasted_source[:,3]
        
        #prepare data into 7D 
        centroid_source = centeroidnp(raycasted_source)
        centroid_target = centeroidnp(raycasted_target)
        centroid_source_flow = centeroidnp(raycasted_source_flow)
        
        source_7d = create_7D(raycasted_source, centroid_source, centroid_source_flow)
        target_7d = create_7D(raycasted_target, centroid_source, centroid_target)
        flow_source_7d = create_7D(raycasted_source_flow, centroid_source, centroid_source_flow)

        flow = flow_source_7d[:,:3]-source_7d[:,:3]
        
    
        #find the biomechanical points in every vertebra 
        surface1 = np.copy(raycasted_source[:,3])
        L1 = raycasted_source[np.argwhere(surface1 == 1.).squeeze()]
        L2 = raycasted_source[np.argwhere(surface1 == 2.).squeeze()]
        L3 = raycasted_source[np.argwhere(surface1 == 3.).squeeze()]
        L4 = raycasted_source[np.argwhere(surface1 == 4.).squeeze()]
        L5 = raycasted_source[np.argwhere(surface1 == 5.).squeeze()]
        
        #create the .npz file
        np.savez_compressed(pa + "raycasted_"+ target_id + ".npz",
                                    flow=flow,
                                    pc1=source_7d,
                                    pc2=target_7d,
                                    cstPts=np.array([L1.shape[0]-1,
                                                    L2.shape[0]-1,L2.shape[0]-2,
                                                    L3.shape[0]-1,L3.shape[0]-2,
                                                   L4.shape[0]-1,L4.shape[0]-2,
                                                   L5.shape[0]-1]))
#         return


## uncomment the cell below and change paths accordingly to create files from full spine data

In [111]:
path_to_npz = "/Users/janelameski/Desktop/jane/sofa/SOFAZIPPED/install/bin/" + "txtFiles/"

for i in range(10):
    write_source_target_flow_as_npz(path_to_npz, i+1)

In [91]:
# data = np.load(path_to_npz+"spine7_10_0.npz")

# np.savetxt("source_pc_npz.txt", data["pc1"])

## Uncomment the cell below and change paths accordingly to create npz training files from raycasted data

In [16]:
# path_to_npz = "/Users/janelameski/Desktop/jane/sofa/SOFAZIPPED/install/bin/" + "txtFiles/"
# path_to_label_maps = "/Users/janelameski/Desktop/labelMaps/"

# for i in range(10):
#     combine_vertebrae_with_label(path_to_npz, i+1, path_to_label_maps)

In [114]:
path_to_npz = "/Users/janelameski/Desktop/jane/sofa/SOFAZIPPED/install/bin/" + "txtFiles/"
path_to_label_maps = "/Users/janelameski/Desktop/labelMaps/"

for i in range(10):
    combine_vertebrae_with_label(path_to_npz, i+1, path_to_label_maps)

In [110]:
data = np.load(path_to_npz+"raycasted_spine5_0_0.npz")

np.savetxt("s_f_pc_npz.txt", data["pc1"][:,:3] + data["flow"])